# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, json, math, time
from tqdm.notebook import tqdm_notebook
import tmdbsimple as tmdb 

# Load API

In [2]:
import json
with open('/Users/Justice/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['api-key'])

In [3]:
import tmdbsimple as tmdb
tmdb.API_KEY =  login['api-key']

# Custom Functions

## Movie Rating Function

In [4]:
def get_movie_with_rating(movie_id):
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    # save the .info .releases dictionaries
    movie_info = movie.info()
    
    releases = movie.releases()
    # Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation==US
        if c['iso_3166_1' ] =='US':
            ## save a "certification" key in the info dict with the certification
            movie_info['certification'] = c['certification']
    return movie_info

## Results to JSON Function

In [5]:
def write_json(new_data, filename): 
    """Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""    
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

# Test API

In [6]:
# Make a movie object using the .Movies function from tmdb
# movie1 = tmdb.Movies('tt0848228')
# movie2 = tmdb.Movies('tt0332280')

In [7]:
# info1 = movie1.info()
# info1

In [8]:
# info2 = movie2.info()
# info2

In [9]:
# print(info1['budget'])
# print(info2['budget'])

In [10]:
# print(info1['revenue'])
# print(info2['revenue'])

In [11]:
# print(info1['imdb_id'])
# print(info2['imdb_id'])

In [12]:
# movie1 = tmdb.Movies('tt0848228')
# info = movie1.info()
# print(info['budget'])

# movie2 = tmdb.Movies('tt0332280')
# info = movie2.info()
# print(info['budget'])

In [13]:
# releases = movie1.releases()
# for c in releases['countries']:
#     if c['iso_3166_1'] == 'US':
#         print(c['certification'])
        
        
# releases = movie2.releases()
# for c in releases['countries']:
#     if c['iso_3166_1'] == 'US':
#         print(c['certification'])        

# Steps before API Call

## Data Folder

In [14]:
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'title.akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz']

## Focused API Years

In [15]:
YEARS_TO_GET = [2000,2001]

# API Call

In [16]:
# Load in the dataframe from project part 1 as basics:
basics = pd.read_csv('Data/title_basics.csv.gz')

In [17]:
# Define errors list
errors = [ ]

In [18]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET,desc='YEARS',
                          position=0):
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'

    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)


    # If it does not exist: create it
    if file_exists == False:
    # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)



    #Saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()
    # saving movie ids to list
    movie_ids = df['tconst'].copy()#.to_list()


    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)


    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]


    #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        # Attempt to retrieve then data for the movie id
        try:
            temp = get_movie_with_rating(movie_id)  #This uses your pre-made function
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)

        # If it fails,  make a dict with just the id and None for certification.
        except Exception as e:
            errors.append([movie_id, e])


    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1461 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1579 [00:00<?, ?it/s]